In [24]:
import pandas as pd
import numpy as np

In [25]:
data = pd.read_csv("../input/Combined_News_DJIA.csv")

I made this a binary classification task. Hence, there are only two labels:

"1" when DJIA Adj Close value rose or stayed as the same;

"0" when DJIA Adj Close value decreased.

-

We want +1 and -1 as labels:


In [26]:
data = data.fillna("")
data.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

In [27]:
new_data

{'Date': [], 'Label': [], 'News': []}

In [28]:
new_data = {'Date': [],'Label': [],'News':[]}

In [31]:
from datetime import datetime
import nltk
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.tokenize import word_tokenize 
import string
from nltk.corpus import stopwords  
from textblob import TextBlob, Word
from nltk.stem import WordNetLemmatizer
from word_forms.lemmatizer import lemmatize
import spacy 

# ps = SnowballStemmer("english") 
# ps = PorterStemmer()
# wordnet_lemmatizer = WordNetLemmatizer()
# nlp=spacy.load('en_core_web_sm')

stop_words = set(stopwords.words('english')) 

for index, row in data.iterrows():
    combined_new = ""
    label = 1
    for column in row.iloc[2:]:
        combined_new+=" "+column[2:-1]
    if row["Label"] == 0:
        label = -1
    combined_new=combined_new.lower()
    combined_new=combined_new.translate(str.maketrans('', '', string.punctuation))
    words = word_tokenize(combined_new)
   
    stemmed_words = []

    for word in words:
        try:
#             stemmed_word = wordnet_lemmatizer.lemmatize(word)
#             stemmed_word = nlp(word)[0].lemma_
            stemmed_word = lemmatize(word)
            if len(stemmed_word) == len(word):
                stemmed_word = word
        except:
            stemmed_word = word
        if len(word) > 3 and stemmed_word not in stop_words and word.isalpha():
            stemmed_words.append(stemmed_word)
    new_data["Date"].append(row["Date"])
    new_data["Label"].append(label)
    new_data["News"].append(stemmed_words)

In [32]:
len(new_data["Date"])

3978

In [33]:
stemmed_words

['woman',
 'mexico',
 'city',
 'final',
 'receive',
 'birth',
 'certify',
 'die',
 'hour',
 'late',
 'trinidad',
 'alvarez',
 'lira',
 'wait',
 'year',
 'proof',
 'born',
 'chief',
 'back',
 'athens',
 'permanent',
 'olympic',
 'preside',
 'france',
 'say',
 'brexit',
 'donald',
 'trum',
 'itish',
 'must',
 'give',
 'police',
 'hour',
 'notice',
 'threaten',
 'hunger',
 'strike',
 'subject',
 'sex',
 'risk',
 'order',
 'despite',
 'never',
 'convict',
 'crime',
 'nobel',
 'laureate',
 'urge',
 'greenpeace',
 'stop',
 'oppose',
 'azil',
 'huge',
 'spike',
 'numbs',
 'police',
 'kill',
 'ahead',
 'olympic',
 'stria',
 'high',
 'court',
 'annul',
 'preside',
 'elect',
 'narrow',
 'lost',
 'rightwing',
 'candidate',
 'cebook',
 'win',
 'privacy',
 'case',
 'track',
 'belgian',
 'want',
 'doesnt',
 'matter',
 'internet',
 'use',
 'log',
 'facebook',
 'itzerland',
 'deny',
 'muslim',
 'girl',
 'citizen',
 'refuse',
 'swim',
 'boy',
 'school',
 'long',
 'consider',
 'naturalise',
 'citizen',


In [34]:
df = pd.DataFrame(new_data, columns = ['Date', 'Label','News'])
len(df)

3978

In [35]:
df.head()

,Date,Label,News
0,2008-08-08,-1,"[georgia, russian, warplane, country, move, br..."
1,2008-08-11,1,"[wont, america, nato, help, wont, help, help, ..."
2,2008-08-12,-1,"[remember, adore, sang, open, ceremony, fake, ..."
3,2008-08-13,-1,"[refuse, israel, weapon, attack, iran, report,..."
4,2008-08-14,1,"[expert, admit, legalise, drug, south, osetia,..."


Check if theres no 0 in data

In [38]:
df.isnull().sum()

Date     0
Label    0
News     0
dtype: int64

In [39]:
preprocessed_data = df.sample(frac=1).reset_index(drop=False)

In [40]:
# train / testsplit with 80/20%
train_set =preprocessed_data.loc[0:len(preprocessed_data)*0.8] 
test_set = preprocessed_data.loc[len(preprocessed_data)*0.8:] 

In [41]:
train_set.to_pickle("train.pkl")
test_set.to_pickle("test.pkl")